### Importações

In [ ]:
import pandas as pd
import os
from google.colab import drive
import joblib

# Importações de ML
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, fbeta_score, make_scorer

# Importações de Reamostragem
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

### Montar o Google Drive


In [ ]:
drive.mount('/content/drive')
output_path = '/content/drive/MyDrive/PS-Ligia_Time16/'
if not os.path.exists(output_path):
    os.makedirs(output_path)

### Carregar Datasets


In [ ]:
'''
Se você executou o notebook de EDA deste projeto, e portanto, gerou e salvou os
datasets processados; Descomente este código abaixo e fique a vontade para
utilizar os datasets que foram salvos no seu próprio Google Drive.

Caso não tenha criado os datasets, utilize o trecho descomentado para carregar
ele já pronto do nosso Drive.
'''


# datasets = {
#     "Base": pd.read_csv(os.path.join(output_path, 'risco_cardiovascular_base.csv')),
#     "Features": pd.read_csv(os.path.join(output_path, 'risco_cardiovascular_features.csv'))
# }

datasets = {
    "Base": pd.read_csv("https://drive.google.com/uc?id=1CT0vw85_nr4SQb1M_M9gLwooFePXZ6FN"),
    "Features": pd.read_csv("https://drive.google.com/uc?id=142P-9xy-kRboRVpHj2t9OniS0DUF0QTp")
}


### Definição de Modelos e Hiperparâmetros para GridSearch

In [ ]:
'''
Aqui criamos o grid de todos modelos com seus respectivos hiperparametros que
vão ser combinados no gridsearch para acharmos a melhor configuração.

Com a pipeline do imblearn tambem é possivel adicionar resampler ou qualquer
pré-processamento pro grid

'''
resampler = [
    SMOTE(random_state=42),
    RandomOverSampler(random_state=42),
    RandomUnderSampler(random_state=42),
    'passthrough'
]

models_config = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'resampler': resampler,
            'classifier__n_estimators': [50, 100, 200],
            'classifier__max_depth': [None, 5, 10],
            'classifier__min_samples_split': [2, 5]
        }
    },
    'DecisionTree': {
        'model': DecisionTreeClassifier(random_state=42),
        'params': {
            'resampler': resampler,
            'classifier__max_depth': [None, 3, 5, 10],
            'classifier__criterion': ['gini', 'entropy'],
            'classifier__min_samples_leaf': [1, 2, 4]
        }
    }
}

### Loop de Treinamento


In [ ]:
model_save_path = os.path.join(output_path, 'RandomForest_DecisionTree_Models')
os.makedirs(model_save_path, exist_ok=True)

results_list = []

f2_scorer = make_scorer(fbeta_score, beta=2, average='macro', pos_label=None)

for ds_name, df in datasets.items():
    print(f"\n>>> Processando Dataset: {ds_name}")

    # Separar X e y
    X = df.drop('BP_Category', axis=1)
    y = df['BP_Category']

    bmi_order = ["Underweight", "Normal Weight", "Overweight", "Obese"]

    # Identificar colunas
    cat_cols = X.copy().drop('BMI Category', axis=1).select_dtypes(include=['object']).columns.tolist()
    num_cols = X.select_dtypes(exclude=['object']).columns.tolist()

    # Preprocessor
    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ("ordinal", OrdinalEncoder(categories=[bmi_order]), ['BMI Category']),
    ])

    # Split Treino/Teste (80/20)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    for model_name, config in models_config.items():
        print(f"\n--- Iniciando GridSearch para {model_name} ---")

        # Pipeline base com placeholder para o resampler
        pipeline = ImbPipeline(steps=[
            ('preprocessor', preprocessor),
            ('resampler', SMOTE()), # Placeholder que será substituído pelo Grid
            ('classifier', config['model'])
        ])

        # Cross-Validation (10 Folds para dataset pequeno)
        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

        # Grid Search
        grid = GridSearchCV(
            pipeline,
            config['params'],
            cv=cv,
            scoring={'f2_macro': f2_scorer, 'f1_macro': 'f1_macro', 'accuracy': 'accuracy'},
            refit='f2_macro',
            return_train_score=True,
            n_jobs=-1,
            verbose=1,
            error_score='raise'
        )

        grid.fit(X_train, y_train)

        cv_res = pd.DataFrame(grid.cv_results_)

        # Métricas de Treino (do melhor modelo)
        best_idx = grid.best_index_
        train_f2 = cv_res.loc[best_idx, 'mean_train_f2_macro']
        train_f1 = cv_res.loc[best_idx, 'mean_train_f1_macro']
        train_acc = cv_res.loc[best_idx, 'mean_train_accuracy']

        # Métricas de Teste
        y_pred = grid.predict(X_test)
        test_f2 = fbeta_score(y_test, y_pred, beta=2, average='macro')
        test_f1 = f1_score(y_test, y_pred, average='macro')
        test_acc = accuracy_score(y_test, y_pred)

        # Salvar Modelo no Drive
        model_filename = f"{model_name}_{ds_name}.pkl"
        joblib.dump(grid.best_estimator_, os.path.join(model_save_path, model_filename))

        # No results_list.append, adicione:
        results_list.append({
            'Model': model_name,
            'Dataset': ds_name,
            'Best_Params': str(grid.best_params_),
            'Train_F2_Macro': train_f2,
            'Train_F1_Macro': train_f1,
            'Train_Accuracy': train_acc,
            'Test_F2_Macro': test_f2,
            'Test_F1_Macro': test_f1,
            'Test_Accuracy': test_acc,
        })

### Resultados

In [ ]:
df_results = pd.DataFrame(results_list)

metrics_save_path = os.path.join(output_path, "Metrics")
os.makedirs(metrics_save_path, exist_ok=True)

# Criar tabelas separadas por modelo
for model_name in models_config.keys():
    df_model = df_results[df_results['Model'] == model_name].copy()

    # Salvar no Drive
    filename = f"resultado_final_{model_name}.csv"
    df_model.to_csv(os.path.join(metrics_save_path, filename), index=False)

    print(f"\n--- Tabela Final: {model_name} ---")
    display(df_model[['Dataset', 'Train_F1_Macro', 'Train_Accuracy', 'Test_F1_Macro', 'Test_Accuracy', 'Best_Params']])

print(f"\nProcesso concluído! Os arquivos foram salvos em: {metrics_save_path}")

### Feature Importance (Exclusivo do RandomForest e DecisionTree)

In [ ]:
# --- Célula: Visualização da Importância das Features ---
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

def gerar_graficos_importancia(results_list, model_save_path, output_path):
    """
    Carrega os modelos salvos e gera gráficos de importância das features
    para cada combinação de Modelo e Dataset.
    """
    # Configuração visual
    sns.set_theme(style="whitegrid")

    for res in results_list:
        model_name = res['Model']
        ds_name = res['Dataset']

        # Caminho do modelo salvo no Drive
        path_modelo = os.path.join(model_save_path, f"{model_name}_{ds_name}.pkl")

        if os.path.exists(path_modelo):
            # Carregar o pipeline completo (inclui preprocessor e classifier)
            pipeline = joblib.load(path_modelo)

            # 1. Recuperar nomes das colunas (considerando o One-Hot Encoding)
            # O get_feature_names_out() extrai os nomes gerados pelo ColumnTransformer
            preprocessor = pipeline.named_steps['preprocessor']
            feature_names = preprocessor.get_feature_names_out()

            # Limpar nomes (remover prefixos técnicos 'num__' e 'cat__')
            feature_names = [name.split('__')[-1] for name in feature_names]

            # 2. Extrair importâncias do classificador (RF ou DT)
            classifier = pipeline.named_steps['classifier']
            importances = classifier.feature_importances_

            # 3. Criar DataFrame para organizar o Top 10
            df_imp = pd.DataFrame({
                'Feature': feature_names,
                'Importance': importances
            }).sort_values(by='Importance', ascending=False).head(10)

            # 4. Criação do Gráfico
            plt.figure(figsize=(10, 6))
            cores = 'viridis' if model_name == 'RandomForest' else 'magma'

            ax = sns.barplot(
                data=df_imp,
                x='Importance',
                y='Feature',
                palette=cores,
                hue='Feature',
                legend=False
            )

            # Estilização
            plt.title(f"Top 10 Features: {model_name}\nDataset: {ds_name}", fontsize=14, fontweight='bold', pad=15)
            plt.xlabel("Importância Relativa (Gini Importance)", fontsize=12)
            plt.ylabel("Atributos", fontsize=12)

            # Adicionar os valores numéricos nas pontas das barras para precisão
            for i in ax.containers:
                ax.bar_label(i, fmt='%.3f', padding=5)

            plt.tight_layout()

            # Salvar o gráfico no Drive
            nome_arquivo = f"feat_imp_{model_name}_{ds_name}.png"
            plt.savefig(os.path.join(metrics_save_path, nome_arquivo), dpi=300)
            plt.show()
            print(f"✅ Gráfico de importância para {model_name} ({ds_name}) salvo com sucesso.")
        else:
            print(f"⚠️ Modelo {model_name}_{ds_name}.pkl não encontrado no caminho especificado.")

# Execução automática
gerar_graficos_importancia(results_list, model_save_path, output_path)